<a href="https://colab.research.google.com/github/benjaminsw/Generalised_Linear_Models/blob/main/R_cars_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
if (!require('gsheet')) install.packages('gsheet'); library('gsheet')
if (!require('ggplot2')) install.packages('ggplot2'); library('ggplot2')
if (!require('gridExtra')) install.packages('gridExtra'); library('gridExtra')
if (!require('nnet')) install.packages('nnet'); library('nnet')
if (!require('sjPlot')) install.packages('sjPlot'); library('sjPlot')
if (!require('ggeffects')) install.packages('ggeffects'); library('ggeffects')
if (!require('MASS')) install.packages('MASS'); library('MASS')

# set plot size
options(repr.plot.width=16, repr.plot.height=8)

Loading required package: sjPlot

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘sjPlot’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘nloptr’, ‘lme4’, ‘sjstats’


Warning message in install.packages("sjPlot"):
“installation of package ‘nloptr’ had non-zero exit status”
Warning message in install.packages("sjPlot"):
“installation of package ‘lme4’ had non-zero exit status”
Warning message in install.packages("sjPlot"):
“installation of package ‘sjstats’ had non-zero exit status”
Warning message in install.packages("sjPlot"):
“installation of package ‘sjPlot’ had non-zero exit status”


ERROR: ignored

In [ ]:
url <- 'https://docs.google.com/spreadsheets/d/1XQqBJwWZnFIeADsmr7bBXDY0hS49GaxEqypM8bgGlYc/edit?usp=sharing'
cars <- gsheet2tbl(url)
cars$resp <- factor(cars$response, levels=c("no/little", "important", "very important"))
cars

In [ ]:
g_legend <- function(a.gplot){
  tmp <- ggplot_gtable(ggplot_build(a.gplot))
  leg <- which(sapply(tmp$grobs, function(x) x$name) == "guide-box")
  legend <- tmp$grobs[[leg]]
  return(legend)}

p1 <- ggplot(cars, aes(x = cars$age, y = cars$frequency, fill = response)) +
  geom_bar(stat = "identity", position = "dodge")+
  xlab("Age groups") + ylab("Frequency") +
  theme(legend.position = "none")

  p2 <- ggplot(cars, aes(x = cars$sex, y = cars$frequency, fill = response)) +
  geom_bar(stat = "identity", position = "dodge")+
  xlab("Sex") + ylab("Frequency") +
  scale_fill_discrete(name = "Response category") + 
  theme(legend.position = "bottom")

carsLeg <- g_legend(p2)
grid.arrange(arrangeGrob(p1 + theme(legend.position="none"),
                         p2 + theme(legend.position="none"),
                         nrow=1), nrow = 2, carsLeg, heights=c(10, 1))

In [ ]:
m1 <- multinom(resp~age+sex, weight=frequency, data=cars)
summary(m1)

In [ ]:
# Null model:
m0 <- multinom(resp~1, weight=frequency, data=cars)
summary(m0)

In [ ]:
# Plot of coefficient estimates on odds ratio scale:
plot_model(m1)

In [ ]:
# Plot of predicted probabilities (fitted values):
p1 <- ggpredict(m1, c("sex", "age"))
plot(p1)

In [ ]:
m2 <- multinom(resp~age*sex, weight=frequency, data=cars)
summary(m2)

In [ ]:
anova(m1,m2)

In [ ]:
# Linear term for age:
cars$agelin <- 0
cars$agelin[cars$age=="24-40"] <- 1
cars$agelin[cars$age=="> 40"] <- 2
cars

In [ ]:
m3 <- multinom(resp~agelin+sex, weight=frequency, data=cars)
summary(m3)

In [ ]:
plot_model(m3)

In [ ]:
# Fitted values (predicted probabilities):
p3 <- ggpredict(m3, c("sex", "agelin"))
plot(p3)

In [ ]:
m4 <- polr(resp ~ sex + age, data=cars, weight=frequency)
summary(m4)

In [ ]:
# Could also try a model with a linear term for age
m5 <- polr(resp ~ sex + agelin, data=cars, weight=frequency)
summary(m5)

In [ ]:
# Predicted probabilities:
p5 <- ggpredict(m5)
plot(p5)